<a href="https://colab.research.google.com/github/Bainer13/Data_BBVA_Team_1/blob/Rodri/Proyecto_bootcamp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Armado del entorno de trabajo



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [3]:
ls

address_bootcamp.snappy.parquet          phones_bootcamp.snappy.parquet
customer_basics_bootcamp.snappy.parquet  sample_data/
emails_bootcamp.snappy.parquet           spark-3.3.2-bin-hadoop3.tgz


In [4]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [5]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [6]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [7]:
!echo $JAVA_HOME

In [8]:
!pip install -q findspark

In [9]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=0bede9f4aa81b2a6dc12cde9c6f1ee600298abf2c9454c133ebccc268cb550e5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [10]:
from pyspark.sql import SparkSession

In [11]:
import pyspark.sql.functions as F

In [12]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [13]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col

In [14]:
spark = SparkSession.builder.getOrCreate()

#Extraccion de datos

In [15]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

In [123]:
#Lectura de parquets
df_address = spark.read.parquet('/content/address_bootcamp.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/customer_basics_bootcamp.snappy.parquet')
df_email = spark.read.parquet('/content/emails_bootcamp.snappy.parquet')
df_phones = spark.read.parquet('/content/phones_bootcamp.snappy.parquet')

In [124]:
#Join entre clientes y telefonos
df_customer_basics_phones = df_customer_basics.join(df_phones, ['customer_id'], how='left').select(df_customer_basics.customer_id, df_phones.prefix_phone_id, df_phones.phone_area_id, df_phones.cellphone_prefix_id, df_phones.phone_exchange_id, df_phones.phone_line_id, df_phones.phone_id, df_phones.last_change_date)
df_customer_basics_phones.sort(F.asc("customer_id")).show(10)

+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+
|customer_id|prefix_phone_id|phone_area_id|cellphone_prefix_id|phone_exchange_id|phone_line_id|     phone_id|last_change_date|
+-----------+---------------+-------------+-------------------+-----------------+-------------+-------------+----------------+
|   00000442|             54|          297|                 15|              529|         6284| 712455962424|      2022-07-20|
|   00000442|             54|          297|                 15|              472|         9337| 384498667593|      2022-07-20|
|   00001419|             54|           11|                 15|             6679|         3207| 044982237001|      2022-11-01|
|   00001939|             54|           11|                 15|             3870|         0150| 008960137788|      2022-09-21|
|   00002575|           null|         null|               null|             null|         null|         null|  

In [125]:
#Join entre clientes y email
df_customer_basics_email = df_customer_basics.join(df_email,['customer_id'], how='left').select(df_customer_basics.customer_id, df_email.email_desc, df_email.last_change_date)
df_customer_basics_email.show()

+-----------+--------------------+----------------+
|customer_id|          email_desc|last_change_date|
+-----------+--------------------+----------------+
|   07867437|                null|            null|
|   06813266|                null|            null|
|   29604216|                null|            null|
|   28028529|                null|            null|
|   28366580|                null|            null|
|   02746969|                null|            null|
|   23441482|                null|            null|
|   23610540|                null|            null|
|   25935956|                null|            null|
|   25848977|                null|            null|
|   01957163|                null|            null|
|   06338965|                null|            null|
|   29676364|                null|            null|
|   26205399|   DIONISI@YAHOO.COM|      2019-12-27|
|   03933825|DIGITALIZACION@EE...|      1900-01-01|
|   29416245|                null|            null|
|   29497813

In [126]:
#join entre clientes y direccion
df_customer_basics_address = df_customer_basics.join(df_address,['customer_id'], how='left').select(df_customer_basics.customer_id, 'street_name', 'address_outdoor_id','address_indoor_id', 'indoor_number', 'address_town_name', 'address_department_name', 'zipcode_id', df_address.last_change_date)
df_customer_basics_address.show()

+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+
|customer_id|         street_name|address_outdoor_id|address_indoor_id|indoor_number|   address_town_name|address_department_name|zipcode_id|last_change_date|
+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+
|   07867437|ALTE FRANCISCO SE...|           0001971|         PB      |      3      |CIUDAD AUTONOMA B...|   CAPITAL FEDERAL  ...|     01416|      2009-01-26|
|   06813266|EL TORDO         ...|           0000866|                 |             |ITUZAINGO        ...|   ITUZAINGO        ...|     01714|      2009-01-26|
|   29604216|                null|              null|             null|         null|                null|                   null|      null|            null|
|   28028529|AMENEDO          ...|           0

In [127]:
df_address.show() #Bloque para visualizar DFs

+-----------+-----------------------+-------------------+--------------+--------------------+------------------+---------------------------+-----------------+-------------+---------------------+--------------------+-----------------------+----------+---------------+-----------+------------------+----------------------+-------------------------+------------------+---------------------+------------------------------+-----------------------+--------------------+---------------------+------------------+-------------------------+-------------------------+------------------+-----------------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+-----------------+---------------------+
|customer_id|add

In [128]:
#Filtrado y reduccion de datos --> PHONES
w = Window.partitionBy(df_customer_basics_phones['customer_id']).orderBy("last_change_date")

In [129]:
df_customer_basics_phones = df_customer_basics_phones.withColumn("row_num", row_number().over(w))

In [130]:
df_customer_basics_phones = df_customer_basics_phones.filter(col("row_num") <= 3)

In [131]:
df_customer_basics_phones = df_customer_basics_phones.withColumn('Numero_Telefono', F.concat_ws("-", df_phones.prefix_phone_id, df_phones.phone_area_id, df_phones.cellphone_prefix_id, df_phones.phone_exchange_id, df_phones.phone_line_id))
df_customer_basics_phones = df_customer_basics_phones.select('customer_id', 'Numero_Telefono', 'phone_id', 'last_change_date')
df_customer_basics_phones.show(10)

+-----------+-------------------+-------------+----------------+
|customer_id|    Numero_Telefono|     phone_id|last_change_date|
+-----------+-------------------+-------------+----------------+
|   00000442| 54-297-15-529-6284| 712455962424|      2022-07-20|
|   00000442| 54-297-15-472-9337| 384498667593|      2022-07-20|
|   00001419| 54-11-15-6679-3207| 044982237001|      2022-11-01|
|   00001939| 54-11-15-3870-0150| 008960137788|      2022-09-21|
|   00002575|                   |         null|            null|
|   00002707|   54-299--447-7116|   6116490163|      2022-08-17|
|   00002790|   54-342--488-3620|   4330463819|      2022-10-08|
|   00002790| 54-342-15-466-2478| 051880599267|      2022-11-18|
|   00004287|54-3541-15-5988-799|1456900382957|      2022-08-29|
|   00004724| 54-11-15-2408-4447| 820582605809|      2022-11-04|
+-----------+-------------------+-------------+----------------+
only showing top 10 rows



In [132]:
df_customer_basics_phones.count()

2070

In [133]:
#Filtrado y reduccion de datos --> EMAILS
w = Window.partitionBy(df_customer_basics_email['customer_id']).orderBy("last_change_date")

In [134]:
df_customer_basics_email = df_customer_basics_email.withColumn("row_num", row_number().over(w))

In [135]:
df_customer_basics_email = df_customer_basics_email.filter(col("row_num") <= 3)

In [136]:
df_customer_basics_email.show(10)

+-----------+--------------------+----------------+-------+
|customer_id|          email_desc|last_change_date|row_num|
+-----------+--------------------+----------------+-------+
|   00000442|                null|            null|      1|
|   00001419|EMILIA.RUBIANES@H...|      2015-10-08|      1|
|   00001419|  JU_LY1@HOTMAIL.COM|      2019-05-11|      2|
|   00001939|                null|            null|      1|
|   00002575|                null|            null|      1|
|   00002707|                null|            null|      1|
|   00002790|DIGITALIZACION@EE...|      1900-01-01|      1|
|   00002790| NOTIENE@HOIMAIL.COM|      2019-07-29|      2|
|   00004287|                null|            null|      1|
|   00004724|                null|            null|      1|
+-----------+--------------------+----------------+-------+
only showing top 10 rows



In [137]:
df_customer_basics_email.count()

1667

In [138]:
#Filtrado y reduccion de datos --> EMAILS
w = Window.partitionBy(df_customer_basics_address['customer_id']).orderBy("last_change_date")

In [139]:
df_customer_basics_address = df_customer_basics_address.withColumn("row_num", row_number().over(w))

In [140]:
df_customer_basics_address = df_customer_basics_address.filter(col("row_num") <= 3)

In [141]:
df_customer_basics_address.show(10)

+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+-------+
|customer_id|         street_name|address_outdoor_id|address_indoor_id|indoor_number|   address_town_name|address_department_name|zipcode_id|last_change_date|row_num|
+-----------+--------------------+------------------+-----------------+-------------+--------------------+-----------------------+----------+----------------+-------+
|   00000442|AGUERO           ...|           0000842|         PB      |      B      |CAPITAL FEDERAL  ...|   112233114455     ...|     01171|      2008-04-21|      1|
|   00001419|ALTE G BROWN     ...|           0001074|         1       |      A      |MORON            ...|   MORON            ...|     01708|      2009-01-26|      1|
|   00001939|JOSE HERNANDEZ   ...|           0002045|         1       |      C      |CIUDAD AUTONOMA B...|   CAPITAL FEDERAL  ...|     01426|      2009-01-26|      1

In [142]:
df_customer_basics_address.count()

1660